In [22]:
from fastcore.all import *
from litesearch import *
from ddgs import DDGS
from lisette import *
from toolslm.funccall import *
from toolslm.md_hier import *
import ast
from ast import get_source_segment as gs
import sys
from embed_anything import ColbertModel
import embed_anything

In [23]:
def is_func(n): return isinstance(n,(ast.FunctionDef,ast.AsyncFunctionDef,ast.ClassDef))
def is_assign(n): return isinstance(n, (ast.Assign, ast.AnnAssign)) and n.value
def is_p_mod(n): return getattr(getattr(n,'parent',None),'__class__',None) == ast.Module
def is_global(n): return False if not is_assign(n) else True if is_p_mod(n) else False
def is_allowed(n): return is_func(n) or is_global(n)

def code2chunks(p):
	txt=Path(p).read_text();tree=ast.parse(txt)
	[setattr(c,'parent',n) for n in ast.walk(tree) for c in ast.iter_child_nodes(n)]
	def n2c(n): return dict(content=gs(txt, n).strip(), metadata=dict(path=p))
	return L(ast.walk(tree)).filter(is_allowed).map(n2c)

venv=Path(f'../.venv/lib/python{sys.version_info.major}.{sys.version_info.minor}/site-packages/')
packages = '|'.join(['fastcore', 'fastlite', 'lisette', 'toolslm', 'chonkie', 'model2vec', 'pymupdf'])
def fetch(sf:Path=venv, pkgs=packages): return globtastic(sf, file_glob='*.py',skip_file_glob='_*',folder_re=pkgs)

In [24]:
chunks = fetch().map(code2chunks).filter(true).concat()

In [26]:
model = ColbertModel.from_pretrained_onnx('answerdotai/answerai-colbert-small-v1',path_in_repo='onnx/model.onnx')
embed_anything.embed_files_batch()

CUDAExecutionProvider is not available


In [27]:
embs = []
for c in chunked(chunks,100): embs+=emb_f([ch['content'] for ch in c], batch_size=100)

In [28]:
fe=first(embs)

Let's build a simple search tool which reranks results from DDGS.


In [ ]:
db = setup_db('code.db')
store = db.mk_store()

In [ ]:
def websearch(q: str,               # query to search
              top_k: int = 10,      # number of top results to return
              reranked:bool=False    # rerank results with flashrank
              ):
	'Web search results reranked with flashrank'
	res = dict2obj(DDGS().text(q, max_results=top_k))
	res = res.map(lambda r: dict(text=r.body, id=r.href, meta=dict(title=r.title)))
	if not reranked: return res
	from flashrank import Ranker, RerankRequest
	ranker = Ranker(max_length=512)
	return ranker.rerank(RerankRequest(q, res))

In [ ]:
websearch('function to access github api in python', reranked=True)

In [ ]:
pkgs = ['fastcore', 'fastlite', 'lisette', 'toolslm', 'chonkie', 'model2vec', 'pymupdf']

In [ ]:
chunks.map()

In [ ]:
df=L(d).map(lambda x: L(x.chunks).map(lambda k: k.content())).concat()

In [ ]:
store.insert_all(df)

In [ ]:
ShowDocRenderer(pkg2content)

In [ ]:
docments(pkg2content, full=True)

In [ ]:
for nm in pkgs: store.insert_all(pkg2content(nm))

In [ ]:
enc_d = AutoEmbeddings().get_embeddings(model_nm).embed

In [ ]:
def rag(q:str,  # query to search
        top_k:int=100,  # number of top results to return
        web:bool=False,  # use web search results
        emb_f=enc_d  # embedding function
        ):
	'search through the database to find relevant context to a query.'
	r = db.search(pre(q), emb_f(q).tobytes(), columns=['id', 'content'], lim=top_k, rrf=True)
	if not web: return r
	return r+websearch(q, top_k=top_k)

In [ ]:
res=rag('parallel processing a list with fastcore')

In [ ]:
from pprint import pprint

In [ ]:
pprint(res)

In [ ]:
def run_code(code:str # code to run
             ):
	'''Run code in python interpreter'''
	return python(code)


In [ ]:
sp='''you have access to a rag tool which has all the code in the packages you installed.
you can also use the web search tool to find relevant documents. If there are functions in the code in rag. use that. Give me example code I can use. '''

In [ ]:
c=Chat('gpt-5-mini',sp,temp=1,search='m',tools=[rag])

In [ ]:
r=c('parallel processing of a list in python',think='m',max_steps=5)

In [ ]:
c.hist[-1].content

In [ ]:
create_heading_dict(c.hist[-1].content)